In [1]:
import folium
import geopandas as gpd
import odc.geo
import pandas as pd
from dea_tools.spatial import subpixel_contours
from dep_tools.namers import DepItemPath
from dep_coastlines.MosaicLoader import DeluxeMosaicLoader
from shapely import polygonize


In [200]:
from dea_tools.classification import predict_xr
from dask.distributed import Client
from joblib import load
mask_model = load("../data/model_9Feb_2.joblib")
model = mask_model['model']
columns = mask_model['predictor_columns']
response_column = mask_model['response_column']
codes = mask_model['codes']
these_codes = codes.groupby(response_column).first()

def colormap(value, **kwargs):
    return these_codes.loc[value,'color']

def code_for_name(name):
    return these_codes.reset_index().set_index('code').loc[name, response_column]

def get_model_mask(xr, model=model, columns=columns):
    with Client():
        mask = predict_xr(model, xr[columns], clean=True).Predictions.rio.write_crs(xr.rio.crs)

    return mask

In [320]:
import xrspatial as xs
import xarray as xr
from skimage.measure import label

def expand(bool_xr):
    return xs.focal.mean(bool_xr) > 0

def remove_disconnected_land(certain_land, candidate_land):
    zones = xr.apply_ufunc(label, candidate_land, None, 0,dask="parallelized", kwargs=dict(connectivity=1))
    connected_or_not = xs.zonal_stats(zones, certain_land.astype("int8"), stats_funcs=["max"])
    connected_zones = connected_or_not["zone"][connected_or_not["max"] == 1]
    return candidate_land.where(zones.isin(connected_zones))




In [385]:
itempath = DepItemPath("ls", "coastlines/mosaics-corrected", "0.6.0", "1999_2023", zero_pad_numbers=True)
loader = DeluxeMosaicLoader(itempath=itempath)

id = [(48,14)]
df = pd.DataFrame([0])
df.index = id
df
from odc.algo import mask_cleanup

ds = loader.load(df)
#mask = get_model_mask(ds)
#clean_area = expand(mask != code_for_name('noisy_water'))
consensus = (ds.nir08 > 1280) & (ds.mndwi < 0) & (ds.ndwi < 0)
expanded = expand(consensus)
#consensus.odc.explore()
connected_areas = remove_disconnected_land(consensus, ds.nir08 > 1280.) == 1
land_plus_one = mask_cleanup(connected_areas, mask_filters=[("dilation", 3)])
land_minus_one = mask_cleanup(connected_areas, mask_filters=[("erosion", 2)])

# erosion of 2 here borks e.g. funafuti but is needed for e.g. shoreline of tongatapu
# maybe only erode areas not in consensus land?
# This works for tongatapu but not funafuti
#analysis_zone = mask_cleanup(connected_areas, mask_filters=[("erosion", 2), ("dilation",2)])
# 
suspicious_connected_areas = (ds.nir08 > 1280) & (xs.focal.mean(consensus) == 0)
sca_clean = mask_cleanup(suspicious_connected_areas, mask_filters=[("erosion", 2), ("dilation",2)])
analysis_zone = connected_areas & ~suspicious_connected_areas
# Only expand where there's an edge that's land
analysis_zone = analysis_zone | mask_cleanup(ds.nir08.where(analysis_zone) > 1280., mask_filters=[("dilation", 1)])
analysis_zone = analysis_zone | mask_cleanup(ds.nir08.where(analysis_zone) > 1280., mask_filters=[("dilation", 1)])
analysis_zone = analysis_zone | mask_cleanup(ds.nir08.where(analysis_zone) > 1280., mask_filters=[("dilation", 1)])
analysis_zone = analysis_zone | mask_cleanup(ds.nir08.where(analysis_zone) > 1280., mask_filters=[("dilation", 1)])
analysis_zone = analysis_zone | mask_cleanup(ds.nir08.where(analysis_zone) > 1280., mask_filters=[("dilation", 1)])
analysis_zone = analysis_zone | mask_cleanup(ds.nir08.where(analysis_zone) > 1280., mask_filters=[("dilation", 1)])
analysis_zone = analysis_zone | mask_cleanup(ds.nir08.where(analysis_zone) > 1280., mask_filters=[("dilation", 1)])
analysis_zone = analysis_zone | mask_cleanup(ds.nir08.where(analysis_zone) > 1280., mask_filters=[("dilation", 1)])
#(ds.nir08.where(analysis_zone) > 1280).odc.explore()

In [386]:
from dep_coastlines.raster_cleaning import load_gadm_land, find_inland_areas

def find_inland_2d(water, ocean):
    water_zones = xr.full_like(water, 0, dtype="int16")
    water_zones.values = label(water.astype("int8"), background=0)
    location_by_zone = xs.zonal_stats(
        water_zones, ocean.astype("int8").compute(), stats_funcs=["max"]
    )
    inland_zones = location_by_zone["zone"][location_by_zone["max"] == 0]
    return water_zones.isin(inland_zones)


gadm_land = load_gadm_land(ds)
gadm_ocean = mask_cleanup(~gadm_land, mask_filters = [("erosion", 2)])
water_bool = ds.nir08 < 1280
inland_areas = find_inland_2d(water_bool, gadm_ocean)


In [388]:
# zero here closes off "corners" that azone will not expand into. Need to be careful though!
lines = subpixel_contours(ds.nir08.where(analysis_zone,0).where(~inland_areas), 1280.0).assign(source = "nir08")
#ndwi_lines = subpixel_contours(ds.ndwi.where(analysis_zone)).assign(source="ndwi")
m = lines.explore(color="black")
ds.nir08.where(analysis_zone).odc.explore(map=m)
#mndwi_lines = subpixel_contours(ready.mndwi).assign(source="mndwi")



Loading data into memory using Dask
Operating in multiple z-value, single array mode


In [ ]:

m = lines.explore(color="green")
folium.LayerControl().add_to(m) 

m